In [1]:
################################################################################
# Load dataset and split it into training and test set
################################################################################

import pandas as pd
import os
from tabulate import tabulate

sample_size = 100000

# Load dateset
df_train = pd.read_csv(os.getcwd() + f'/data/sample-{sample_size}-2_train.csv')
df_test = pd.read_csv(os.getcwd() + f'/data/sample-{sample_size}-2_test.csv')

# Split dataset according to attack type and drop columns
normal_df_train = df_train[df_train['attack'] == 0].drop(columns=['attack', 'category', 'subcategory'])
normal_df_test = df_test[df_test['attack'] == 0].drop(columns=['attack', 'category', 'subcategory'])
attack_df_train = df_train[df_train['attack'] == 1].drop(columns=['attack', 'category', 'subcategory'])
attack_df_test = df_test[df_test['attack'] == 1].drop(columns=['attack', 'category', 'subcategory'])

# Print dataset sizes in a table
data = [
    ["Normal", normal_df_train.shape[0] + normal_df_test.shape[0], normal_df_train.shape[0], normal_df_test.shape[0]],
    ["Attack", attack_df_train.shape[0] + attack_df_test.shape[0], attack_df_train.shape[0], attack_df_test.shape[0]]
]
print(tabulate(data, headers=["Atack type", "Total", "Train", "Test"], tablefmt="grid"))

C:\Users\S4025371\AppData\Local\Temp\ipykernel_25872\3661171010.py:12: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(os.getcwd() + f'/data/sample-{sample_size}-2_train.csv')


+--------------+---------+---------+--------+
| Atack type   |   Total |   Train |   Test |
+==============+=========+=========+========+
| Normal       |   50000 |   40000 |  10000 |
+--------------+---------+---------+--------+
| Attack       |   50000 |   40000 |  10000 |
+--------------+---------+---------+--------+


In [ ]:
################################################################################
# Predict from Vector Store - Search Type Similarity (default)
################################################################################

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

train_set_size = 10000

embeddings = HuggingFaceEmbeddings()

vector_store = Chroma(
    collection_name="bot-iot",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

retriever = vector_store.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 10})

datasets = {"normal": normal_df_test, "attack": attack_df_test}
y_pred = []
y_true = []
for attack_type, dataset in datasets.items():
    test_set_size = dataset.shape[0]
    for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries..."):
        query_document = str(dataset.iloc[i].to_list())
        similar_documents = retriever.invoke(query_document, filter={"source": "bot-iot"})
        y_true.append(attack_type)
        y_pred.append(mode([doc.metadata["label"] for doc in similar_documents]))

c_report = classification_report(y_true, y_pred, digits=4)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/vs/result-vs-df-{sample_size}-2-{train_set_size}.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

In [ ]:
################################################################################
# Predict from Vector Store - Search Type MMR
################################################################################

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

train_set_size = sample_size

embeddings = HuggingFaceEmbeddings()

vector_store = Chroma(
    collection_name="bot-iot",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 10, "fetch_k": 10})

datasets = {"normal": normal_df_test, "attack": attack_df_test}
y_pred = []
y_true = []
for attack_type, dataset in datasets.items():
    test_set_size = dataset.shape[0]
    for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries..."):
        query_document = str(dataset.iloc[i].to_list())
        similar_documents = retriever.invoke(query_document, filter={"source": "bot-iot"})
        y_true.append(attack_type)
        y_pred.append(mode([doc.metadata["label"] for doc in similar_documents]))

c_report = classification_report(y_true, y_pred, digits=4)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/vs/result-vs-mr-{sample_size}-2-{train_set_size}.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

In [2]:
################################################################################
# Predict from Vector Store - Mean Vector
################################################################################

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import numpy as np
from sentence_transformers import util

train_set_size = 100000

embeddings = HuggingFaceEmbeddings()

vector_store = Chroma(
    collection_name="bot-iot",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()

datasets = {"normal": normal_df_test, "attack": attack_df_test}
y_pred = []
y_true = []
for attack_type, dataset in datasets.items():
    test_set_size = dataset.shape[0]
    for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries..."):
        query_content = str(dataset.iloc[i].to_list())
        query_embedding = embeddings.embed_query(query_content)
        # Calculate cosine similarity
        # normal_similarity = np.dot(normal_mean_vector, query_embedding) / (np.linalg.norm(normal_mean_vector) * np.linalg.norm(query_embedding))
        # attack_similarity = np.dot(attack_mean_vector, query_embedding) / (np.linalg.norm(attack_mean_vector) * np.linalg.norm(query_embedding))
        normal_similarity = util.cos_sim(query_embedding, normal_mean_vector).float()
        attack_similarity = util.cos_sim(query_embedding, attack_mean_vector).float()
        y_true.append(attack_type)
        y_pred.append("normal" if normal_similarity > attack_similarity else "attack")

c_report = classification_report(y_true, y_pred, digits=4)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/vs/result-vs-mv-{sample_size}-2-{train_set_size}.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Predicting attack entries...: 100%|█████████████████████████| 10000/10000 [1:04:05<00:00,  2.60it/s]


              precision    recall  f1-score   support

      attack     0.8728    0.9871    0.9264     10000
      normal     0.9852    0.8561    0.9161     10000

    accuracy                         0.9216     20000
   macro avg     0.9290    0.9216    0.9213     20000
weighted avg     0.9290    0.9216    0.9213     20000

[[9871  129]
 [1439 8561]]


In [9]:
################################################################################
# Predict from Vector Store - Mean Vector
################################################################################

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import numpy as np
from sentence_transformers import util

train_set_size = 100000
test_set_size = 100000
dataset_name = "bot-iot"

embeddings = HuggingFaceEmbeddings()

# Calculate mean vectors
vector_store = Chroma(
    collection_name=dataset_name,
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()

# Load test dataset
vector_store_test = Chroma(
    collection_name=dataset_name+"-test",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{test_set_size}-2-test")

normal_vectors_test = vector_store_test._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
attack_vectors_teset = vector_store_test._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']

vectors_test = {"normal": normal_vectors_test, "attack": attack_vectors_teset}
y_pred = []
y_true = []
for attack_type, vectors in vectors_test.items():
    for i in tqdm(range(len(vectors)), ncols=100, desc=f"Predicting {attack_type} entries..."):
        query_embedding = vectors[i]
        normal_similarity = util.cos_sim(query_embedding, normal_mean_vector).float()
        attack_similarity = util.cos_sim(query_embedding, attack_mean_vector).float()
        y_true.append(attack_type)
        y_pred.append("normal" if normal_similarity > attack_similarity else "attack")

c_report = classification_report(y_true, y_pred, digits=4)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/vs/result-vs-mv-{train_set_size}-{test_set_size}-new.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Predicting attack entries...: 100%|██████████████████████████| 10000/10000 [00:25<00:00, 395.07it/s]


              precision    recall  f1-score   support

      attack     0.8728    0.9871    0.9264     10000
      normal     0.9852    0.8561    0.9161     10000

    accuracy                         0.9216     20000
   macro avg     0.9290    0.9216    0.9213     20000
weighted avg     0.9290    0.9216    0.9213     20000

[[9871  129]
 [1439 8561]]
